In [ ]:
%%capture
!pip install evaluate sentencepiece accelerator seqeval
!pip uninstall -y tensorflow protobuf
!pip install "tensorflow==2.15.*" "protobuf==3.20.3"

from huggingface_hub import login
login("")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import evaluate
seqeval = evaluate.load("seqeval")

dataset = load_dataset("nqdhocai/phoner_covid19")
label_list = [
    "O",
    "B-AGE", "I-AGE",
    "B-DATE", "I-DATE",
    "B-GENDER", "I-GENDER",
    "B-JOB", "I-JOB",                
    "B-LOCATION", "I-LOCATION",
    "B-NAME", "I-NAME",             
    "B-ORGANIZATION", "I-ORGANIZATION",
    "B-PATIENT_ID", "I-PATIENT_ID",
    "B-SYMPTOM_AND_DISEASE", "I-SYMPTOM_AND_DISEASE",
    "B-TRANSPORTATION", "I-TRANSPORTATION"
]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

model_name = "demdecuong/vihealthbert-base-word"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

MAX_LEN = 256
label_all_tokens = False

def tag_to_id(t):
    if isinstance(t, (int, np.integer)):
        return int(t)
    return label2id[t]   

def tokenize_and_align_labels(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    for words, tags in zip(examples["words"], examples["tags"]):
        tokens = []
        label_ids = []

        for word, tag in zip(words, tags):
            word_tokens = tokenizer.tokenize(word)
            if len(word_tokens) == 0:
                word_tokens = [tokenizer.unk_token]

            if len(tokens) + len(word_tokens) > MAX_LEN - 2:
                break

            tokens.extend(word_tokens)

            tid = tag_to_id(tag)
            label_ids.append(tid)
            label_ids.extend([-100] * (len(word_tokens) - 1))

        cls_tok = tokenizer.cls_token or "[CLS]"
        sep_tok = tokenizer.sep_token or "[SEP]"

        tokens = [cls_tok] + tokens + [sep_tok]
        label_ids = [-100] + label_ids + [-100]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        pad_len = MAX_LEN - len(input_ids)

        if pad_len > 0:
            input_ids += [pad_id] * pad_len
            attention_mask += [0] * pad_len
            label_ids += [-100] * pad_len
        else:
            input_ids = input_ids[:MAX_LEN]
            attention_mask = attention_mask[:MAX_LEN]
            label_ids = label_ids[:MAX_LEN]

        all_input_ids.append(input_ids)
        all_attention_mask.append(attention_mask)
        all_labels.append(label_ids)

    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_mask,
        "labels": all_labels
    }

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="models/phobert_ner_standard",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    lr_scheduler_type="linear",
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    logging_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/72.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9027 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

🚀 Bắt đầu train NER truyền thống...


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,0.891200,0.281874,0.682755,0.761650,0.720048,0.936748
200,0.239100,0.143986,0.904733,0.932935,0.918617,0.976314
300,0.162600,0.107953,0.912270,0.952771,0.932081,0.979106
400,0.119700,0.088241,0.923472,0.961272,0.941993,0.982738
500,0.105700,0.082318,0.928701,0.967884,0.947888,0.983519
600,0.092300,0.077811,0.927928,0.972922,0.949892,0.983549
700,0.079000,0.075396,0.930009,0.974811,0.951883,0.983939
800,0.077000,0.073453,0.932249,0.974811,0.953055,0.984179


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=849, training_loss=0.21263456035699385, metrics={'train_runtime': 808.6134, 'train_samples_per_second': 33.491, 'train_steps_per_second': 1.05, 'total_flos': 3538696499817984.0, 'train_loss': 0.21263456035699385, 'epoch': 3.0})